Basado en el scrapper hecho por México Abierto:
https://github.com/mxabierto/scraper-sniim/blob/master/sniim/precios_historicos.py


In [1]:
# Importar librerías

import re
import sys
import csv
from bs4 import BeautifulSoup
import urllib.request
from urllib.error import HTTPError
import logging

import os, shutil
from datetime import datetime, timedelta

In [21]:
#shutil.rmtree("raw_data")

In [2]:
# Creación de carpetas temporales y de salida
#print(os.getcwd())
subdir1 = 'temp'
subdir2 = 'raw_data'

try:
  if not os.path.exists(subdir1):
    os.makedirs(subdir1)
  if not os.path.exists(subdir2):
    os.makedirs(subdir2)
except Exception as e:
        print ("Error al crear carpeta: ", e)


C:\Users\Janneth\Documents\MCD\Semestre1\Ing. de Características\RedBAMX\descarga


In [18]:
# Función que realiza el scrapping para extraer una tabla de información y  guardarla en archivo CSV
def creaTabla(url, archivo_salida, paginacion, categoria):
    print (url, archivo_salida, paginacion, categoria)
    try:

        with urllib.request.urlopen(url+str(paginacion), timeout=500) as response, \
          open('temp/'+archivo_salida, 'wb') as out_file, \
          open('raw_data/'+archivo_salida+".csv", 'w', newline="\n") as csvfile:

            data = response.read().decode('utf-8').encode('utf-8')
            soup = BeautifulSoup(data)

            #print(soup.find('span', attrs={"id":"lblPaginacion"}))
            #print(soup.find('span', attrs={"id":"lblPaginacion"}).get_text())

            try:
              #print(soup.find('span', attrs={"id":"lblPaginacion"}).get_text())
              paginas = soup.find('span', attrs={"id":"lblPaginacion"}).get_text()
              #print(paginas)
              print(paginas, paginas != 'Página  1 de  1')



              total_paginas = paginas.split(' ')[-1]
              #print(total_paginas)
              if total_paginas != '1':
                  #print("totalpag dif 1: ", paginacion * int(total_paginas))
                  return creaTabla(url, archivo_salida, paginacion * int(total_paginas), categoria)
              out_file.write(data)


              table = soup.find('table',attrs={"id":"tblResultados"})

              spamwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

              for row in table.find_all("tr"):
                  x = [td.get_text() for td in row.find_all("td")]
                  #x.append(categoria)
                  #print(len(x))
                  #print(x)
                  spamwriter.writerow(x)



            except:
              print ('La página no tiene tabla, ', url, paginacion)
              #borrar archivos
              if os.path.exists("raw_data/"+archivo_salida+".csv"):  
                csvfile.close()
                os.remove("raw_data/"+archivo_salida+".csv")
              if os.path.exists("temp/"+archivo_salida):
                out_file.close()
                os.remove("temp/"+archivo_salida)
              pass
    except Exception as e:
      print ("Error: ", url,e)



In [38]:
# Se definen las fechas de búsqueda
str_fecha_inicio = "01/01/2020"
str_fecha_final = "30/12/2023"
max_intervalo_años = 5

# Se convierten a tipo date
dt_fecha_inicio = datetime.strptime(str_fecha_inicio, '%d/%m/%Y')
dt_fecha_final = datetime.strptime(str_fecha_final, '%d/%m/%Y')

# Ligas a precios
mapa_precios_url = "http://www.economia-sniim.gob.mx/e_MenNal.asp"
base_url = 'http://www.economia-sniim.gob.mx/Nuevo/Consultas/MercadosNacionales/PreciosDeMercado/Agricolas'
frutas_hortalizas_endpoint = "/ResultadosConsultaFechaFrutasYHortalizas.aspx"


with urllib.request.urlopen(mapa_precios_url) as response, \
  open('mapa.aspx', 'wb') as out_file:
    data = response.read() #.decode('utf-8').encode('utf-8') # a `bytes` object
    out_file.write(data)
    soup_directorio = BeautifulSoup(data)

    for anchor in soup_directorio.findAll('a', string=re.compile("Precio\sde\s[^la|los|Granos]\w+")):
        producto_id = re.search(r"ProductoId=(\d+)", anchor['href']).group(1)
        nombre_de_lista = anchor.string
        
        # Si el nombre de lista contiene "/" se reemplaza por "_" para prevenir error al crear archivos con este nombre
        if "/" in nombre_de_lista:
            nombre_de_lista = nombre_de_lista.replace("/", "_")
        
        print(nombre_de_lista)
        '''
        # Prueba descargando por producto
        if nombre_de_lista == "Precio de Limón s_semilla" :
          flag = True
        else:
          flag = False
        if flag:
        '''
        
        # Si el periodo de tiempo es mayor a 5 años, se descarga por segmentos de 5 años para prevenir error en el servidor de datos
        if(dt_fecha_final.year - dt_fecha_inicio.year) > max_intervalo_años:
          for anio_inicio in range(dt_fecha_inicio.year, dt_fecha_final.year, max_intervalo_años):
            # Si el año es mayor que el año de la fecha final, se toma el de la fecha final
            if(anio_inicio + (max_intervalo_años-1)) > dt_fecha_final.year:
              anio_final = dt_fecha_final.year
            else:
              anio_final = anio_inicio + (max_intervalo_años-1)

            url = base_url + frutas_hortalizas_endpoint + f"?ProductoId={producto_id}&fechaInicio=01/01/{anio_inicio}&fechaFinal=31/12/{anio_final}&RegistrosPorPagina="
            #print(url)

            #if(url.find("http:") >= 0):
              #creaTabla(url, nombre_de_lista.replace(" ","_")+"_"+str(anio_inicio)+"-"+str(anio_final), 1000, nombre_de_lista.replace(" ","_"))


        # De lo contrario, si el periodo de tiempo es menor a 5 años, se descarga en un solo segmento
        else:
            url = base_url + frutas_hortalizas_endpoint + f"?ProductoId={producto_id}&fechaInicio={str_fecha_inicio}&fechaFinal={str_fecha_final}&RegistrosPorPagina="
            #print(url)

            #if(url.find("http:") >= 0):
              #creaTabla(url, nombre_de_lista.replace(" ","_")+"_"+str(dt_fecha_inicio.year)+"-"+str(dt_fecha_final.year), 1000, nombre_de_lista.replace(" ","_"))

print("Descarga finalizada.")

Precio de Acelga
Precio de Aguacate Criollo
Precio de Aguacate Fuerte
Precio de Aguacate Hass
Precio de Aguacate Hass adelantado
Precio de Aguacate Hass flor vieja
Precio de Aguacate Pagua
Precio de Ajo Blanco
Precio de Ajo Blanco # 8
Precio de Ajo Morado
Precio de Apio
Precio de Berenjena
Precio de Betabel
Precio de Brócoli
Precio de Cacahuate
Precio de Calabacita Criolla
Precio de Calabacita Italiana
Precio de Calabacita regional
Precio de Calabaza de castilla
Precio de Camote
Precio de Caña
Precio de Cebolla Bola
Precio de Cebolla Bola grande
Precio de Cebolla de rabo
Precio de Cebolla Morada
Precio de Cebolla Roja
Precio de Cereza
Precio de Cilantro
Precio de Ciruela Allende
Precio de Ciruela Amarilla
Precio de Ciruela Claudia
Precio de Ciruela huesuda Amarilla
Precio de Ciruela huesuda Roja
Precio de Ciruela Moscatel
Precio de Ciruela Negra
Precio de Ciruela pasa
Precio de Ciruela Piña
Precio de Ciruela Rayada
Precio de Ciruela Roja
Precio de Coco
Precio de Col grande
Precio de Co